<h1>P2: (12 pts) Gestión de Productos con Python </h1>

In [4]:
import requests
import json
from pymongo import MongoClient

In [5]:
class MongoDBHandler:
    def __init__(self):
        self.mongo_uri = "mongodb://localhost:27017/"
        self.db_name = "utec_store"
        self.categories_name="categorias"
        self.productos_name="productos"
        self.connect_mongo()
    def connect_mongo(self):
        try:
            client = MongoClient(self.mongo_uri, serverSelectionTimeoutMS=5000)
            client.server_info()
            self.db = client[self.db_name]
            print(f"Conectado a la base de datos '{self.db_name}' en MongoDB.")
        except Exception as e:
            print(f"Error al conectar a MongoDB: {e}")
            print("Asegúrese de que MongoDB esté en ejecución en el puerto correcto.")
            exit(1)

    def fetch_json_data(self, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as errh:
            print("HTTP Error:", errh)
        except requests.exceptions.ConnectionError as errc:
            print("Error de conexión:", errc)
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:", errt)
        except requests.exceptions.RequestException as err:
            print("Error desconocido:", err)
        return None

    def fetch_all_categories(self, base_url):
        categories = self.fetch_json_data(base_url)
        if categories:
            for category in categories:
                # Verificamos si category es un string simple o un objeto
                if isinstance(category, dict):
                    # Si es un objeto, guardamos todas sus propiedades
                    self.db[self.categories_name].update_one(
                        {"slug": category.get("slug")},
                        {"$set": category},
                        upsert=True
                    )
                else:
                    # Si es un string simple, guardamos solo el nombre
                    self.db[self.categories_name].update_one(
                        {"name": category},
                        {"$set": {"name": category, "slug": category}},
                        upsert=True
                    )
            print(f"Se cargaron {len(categories)} categorías.")
        else:
            print("No se pudieron obtener las categorías.")

    def fetch_all_products(self):
        categories = list(self.db[self.categories_name].find())
        
        if not categories:
            print("No hay categorías disponibles en la base de datos.")
            return
        
        total_products = 0
        
        for category_doc in categories:

            category_id = category_doc.get("slug", category_doc.get("name", ""))
            
            if isinstance(category_id, str) and ('{' in category_id or "'" in category_id):
                try:
                    category_id = category_id.replace("'", '"')
                    category_data = json.loads(category_id)
                    category_id = category_data.get("slug", "")
                except:
                    category_id = category_doc.get("_id", "")
            
            if not isinstance(category_id, str) or not category_id:
                print(f"Saltando categoría con ID inválido: {category_doc}")
                continue
            
            products_url = f"https://dummyjson.com/products/category/{category_id}"
            print(f"Consultando URL: {products_url}")
            products_data = self.fetch_json_data(products_url)
            
            if products_data and "products" in products_data:
                products = products_data["products"]
                
                for product in products:
                    if "category" not in product:
                        product["category"] = category_id
                    
                    self.db[self.productos_name].update_one(
                        {"id": product["id"]},
                        {"$set": product},
                        upsert=True
                    )
                    total_products += 1
                
                print(f"Se cargaron {len(products)} productos de la categoría '{category_id}'")
            else:
                print(f"No se encontraron productos para la categoría '{category_id}'")
        
        print(f"Se cargaron un total de {total_products} productos de {len(categories)} categorías.")
        
    # Métodos CRUD
    def crear_indice(self, campo):
        try:
            self.db[self.productos_name].create_index(campo)
            print(f"Índice creado en el campo '{campo}'")
        except Exception as e:
            print(f"Error al crear índice: {e}")
            
    def crear_producto(self, producto):
        try:
            result = self.db[self.productos_name].insert_one(producto)
            print(f"Producto creado con ID: {result.inserted_id}")
            return result.inserted_id
        except Exception as e:
            print(f"Error al crear producto: {e}")
            return None
            
    def obtener_productos(self, filtro=None):
        try:
            if filtro is None:
                filtro = {}
            productos = list(self.db[self.productos_name].find(filtro))
            return productos
        except Exception as e:
            print(f"Error al obtener productos: {e}")
            return []
            
    def obtener_producto(self, filtro):
        try:
            producto = self.db[self.productos_name].find_one(filtro)
            return producto
        except Exception as e:
            print(f"Error al obtener producto: {e}")
            return None
            
    def actualizar_producto(self, filtro, actualizacion):
        try:
            result = self.db[self.productos_name].update_one(
                filtro, 
                {"$set": actualizacion}
            )
            print(f"Numero de productos actualizados: {result.modified_count}")
            return result.modified_count
        except Exception as e:
            print(f"Error al actualizar producto: {e}")
            return 0
            
    def eliminar_producto(self, filtro):
        try:
            result = self.db[self.productos_name].delete_one(filtro)
            print(f"Productos eliminados: {result.deleted_count}")
            return result.deleted_count
        except Exception as e:
            print(f"Error al eliminar producto: {e}")
            return 0
            
    # Consultas personalizadas
    def obtener_productos_por_precio(self, precio):
        return self.obtener_productos({"price": precio})
    
    def obtener_productos_por_nombre(self, texto):
        return self.obtener_productos({"title": {"$regex": texto, "$options": "i"}})
    
    # Consultas de agregación
    def precio_promedio(self):
        """Calcular el precio promedio de todos los productos"""
        try:
            pipeline = [
                {"$group": {"_id": None, "promedio": {"$avg": "$price"}}}
            ]
            resultado = list(self.db[self.productos_name].aggregate(pipeline))
            if resultado:
                return resultado[0]["promedio"]
            return 0
        except Exception as e:
            print(f"Error al calcular precio promedio: {e}")
            return 0
    
    def contar_productos(self):
        """Contar el total de productos"""
        try:
            return self.db[self.productos_name].count_documents({})
        except Exception as e:
            print(f"Error al contar productos: {e}")
            return 0
    
    def mayor_stock_categoria(self):
        """Obtener el producto con mayor stock por categoría"""
        try:
            pipeline = [
                {"$group": {
                    "_id": "$category",
                    "producto": {"$first": "$$ROOT"},
                    "max_stock": {"$max": "$stock"}
                }},
                {"$project": {
                    "categoria": "$_id",
                    "producto": 1,
                    "stock_maximo": "$max_stock",
                    "_id": 0
                }}
            ]
            resultado = list(self.db[self.productos_name].aggregate(pipeline))
            return resultado
        except Exception as e:
            print(f"Error al obtener producto con mayor stock por categoría: {e}")
            return []

In [8]:
mongo_handler = MongoDBHandler()

# URL del JSON para las categorías y productos
categories_url = "https://dummyjson.com/products/categories"
# 1. Cargar todas las categorías
mongo_handler.fetch_all_categories(categories_url)

# 2. Cargar todos los productos de cada categoría
mongo_handler.fetch_all_products()

Conectado a la base de datos 'utec_store' en MongoDB.
Se cargaron 24 categorías.
Consultando URL: https://dummyjson.com/products/category/beauty
Se cargaron 5 productos de la categoría 'beauty'
Consultando URL: https://dummyjson.com/products/category/fragrances
Se cargaron 5 productos de la categoría 'fragrances'
Consultando URL: https://dummyjson.com/products/category/furniture
Se cargaron 5 productos de la categoría 'furniture'
Consultando URL: https://dummyjson.com/products/category/groceries
Se cargaron 27 productos de la categoría 'groceries'
Consultando URL: https://dummyjson.com/products/category/home-decoration
Se cargaron 5 productos de la categoría 'home-decoration'
Consultando URL: https://dummyjson.com/products/category/kitchen-accessories
Se cargaron 30 productos de la categoría 'kitchen-accessories'
Consultando URL: https://dummyjson.com/products/category/laptops
Se cargaron 5 productos de la categoría 'laptops'
Consultando URL: https://dummyjson.com/products/category/men

In [9]:
# Probar las nuevas funcionalidades CRUD y de filtrado
mongo_handler = MongoDBHandler()

# Crear índice en campo específico
mongo_handler.crear_indice(campo = "nombre")

# Crear productos
mongo_handler.crear_producto({"nombre": "Producto 1", "price": 100})
mongo_handler.crear_producto({"nombre": "Producto 2", "price": 200})
mongo_handler.crear_producto({"nombre": "Producto 3", "price": 300})

# Leer productos
productos = mongo_handler.obtener_productos()
print("Lista de productos:", productos[0:2])  # Limitamos la salida para no saturar la consola

# Leer producto específico
if productos:
    producto = mongo_handler.obtener_producto({"_id": productos[0]['_id']})
    print("Producto obtenido:", producto)

# Actualizar un producto
if productos:
    mongo_handler.actualizar_producto({"_id": productos[-1]['_id']}, {"precio": 120})

# Eliminar un producto
if productos:
    mongo_handler.eliminar_producto({"_id": productos[-1]['_id']})

# Consultas básicas
print("Productos con precio 100:", mongo_handler.obtener_productos_por_precio(100))
print("Productos que contienen '1' en el nombre:", mongo_handler.obtener_productos_por_nombre("1"))

# Consultas agregadas
print("Precio promedio de productos:", mongo_handler.precio_promedio())
print("Total de productos:", mongo_handler.contar_productos())
print("El producto con mayor stock por categoría:", mongo_handler.mayor_stock_categoria())

Conectado a la base de datos 'utec_store' en MongoDB.
Índice creado en el campo 'nombre'
Producto creado con ID: 6864ba0c58df476f6e127212
Producto creado con ID: 6864ba0c58df476f6e127213
Producto creado con ID: 6864ba0c58df476f6e127214
Lista de productos: [{'_id': ObjectId('6864ba01a0598813f286e999'), 'id': 1, 'availabilityStatus': 'In Stock', 'brand': 'Essence', 'category': 'beauty', 'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.', 'dimensions': {'width': 15.14, 'height': 13.08, 'depth': 22.99}, 'discountPercentage': 10.48, 'images': ['https://cdn.dummyjson.com/product-images/beauty/essence-mascara-lash-princess/1.webp'], 'meta': {'createdAt': '2025-04-30T09:41:02.053Z', 'updatedAt': '2025-04-30T09:41:02.053Z', 'barcode': '5784719087687', 'qrCode': 'https://cdn.dummyjson.com/public/qr-code.png'}, 'minimumOrderQuantity': 48, 'price': 9.99, '